In [ ]:
#Install required libraries
!pip install numpy pandas faiss-gpu torch transformers sentence_transformers rouge --quiet
!pip install rouge-score
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 42.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=da77cf86829b99c9c9b421bbe8be52721a678c524bfb87e9bed3e0f260e9256d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=2adb44993b93d1b6cff8effbd5207f3762d1146e1e45205fe106cb8b14239cb0
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
#Import required libraries
import re
import numpy as np
import pandas as pd
import faiss
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer
from rouge import Rouge
from rouge_score import rouge_scorer

In [ ]:
#Read and display the dataset
movies = pd.read_excel("sample_data/summarized_dataset.xlsx")
movies.head()

,TITLE,Description,PlotSummary
0,21,"Ben Campbell, a mathematics major at the Massa...","Ben Campbell, a mathematics major at the Massa..."
1,10 Things I Hate About You,"Cameron James, a new student at Padua High Sch...",Cameron James becomes smitten with popular sop...
2,101 Dalmatians(1996),American video game designer Roger Dearly live...,American video game designer Roger Dearly live...
3,12 Angry Men,"On a hot summer day, a jury in the New York Co...",An impoverished 18-year-old boy is accused of ...
4,12 Years a Slave,Solomon Northup is a free African-American man...,Solomon Northup is a free African-American man...


In [ ]:
#Preprocess the text with first splitting the sentence into seperate words and set a maximum number of 1024 words
def preprocess_text(text, max_words=1024):
    if isinstance(text, float) and pd.isnull(text):
        return ''

    # Split text into words and limit to max_words
    words = text.split()[:max_words]

    # Join words back into a single string
    text = ' '.join(words)

    # Remove newline, carriage return, and tab characters
    regex = re.compile(r'[\n\r\t]')
    text = regex.sub(" ", text)

    # Remove extra spaces
    text = re.sub(' +', ' ', text).strip()

    return text

In [ ]:
movies['Description'] = movies['Description'].apply(preprocess_text)
movies['PlotSummary'] = movies['PlotSummary'].apply(preprocess_text)

In [ ]:
#Size of the dataset
movies.shape

(1287, 3)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
#Data for training (1110 591)
plots = []
for i in movies.Description:
    try:
        k = i.split(" ")
        plots.append(i)
    except:
        continue

In [ ]:
movies = movies[movies["Description"].isin(plots)]

In [ ]:
plot_list = movies.Description.tolist()

def split_list(lst, batch):
    for i in range(0, len(lst), batch):
        yield lst[i:i + batch]

batch = 32
batches = list(split_list(plot_list, batch))

In [ ]:
#BART model-Bidirectional and Auto-Regressive Transformer
model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-3')
model.to(device)
model.eval()
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-3')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [ ]:
#Generate summaries using BART for the dataset
all_summaries = []
for batch in batches:
    with torch.no_grad():
        inputs = tokenizer(batch,
                           max_length=1024,
                           padding=True,
                           truncation=True,
                           return_tensors='pt')

        summary_ids = model.generate(inputs['input_ids'].to(device), max_length=128, early_stopping=True).to('cpu')

        all_summaries+= [txt.strip() for txt in tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)]
        del inputs, summary_ids
        torch.cuda.empty_cache()

movies['Plot Summary(bart)'] = all_summaries

In [ ]:
#Save generated summaries into an excel file
movies=movies.to_excel('generated_summaries.xlsx', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

#Display the generated summaries
op=pd.read_excel("/content/drive/My Drive/Research/generated_bartsummaries.xlsx")
op.head()

Mounted at /content/drive


,TITLE,Description,PlotSummary,Plot Summary(bart)
0,21,"Ben Campbell, a mathematics major at the Massa...","Ben Campbell, a mathematics major at the Massa...","Ben Campbell, a mathematics major at the Massa..."
1,10 Things I Hate About You,"Cameron James, a new student at Padua High Sch...",Cameron James becomes smitten with popular sop...,Michael Eckman warns sophomore Bianca Stratfor...
2,101 Dalmatians(1996),American video game designer Roger Dearly live...,American video game designer Roger Dearly live...,American video game designer Roger Dearly live...
3,12 Angry Men,"On a hot summer day, a jury in the New York Co...",An impoverished 18-year-old boy is accused of ...,The 18-year-old boy is accused of killing his ...
4,12 Years a Slave,Solomon Northup is a free African-American man...,Solomon Northup is a free African-American man...,Slave trader Theophilus Freeman gives Northup ...


In [ ]:
#Generate summaries for movies not in the dataset by referring to wikipedia and using t5 model
import wikipedia
import requests
from bs4 import BeautifulSoup
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
from nltk.tokenize import sent_tokenize

# Initialize NLTK tokenizer and download punkt if necessary
nltk.download('punkt')

# Initialize T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Initialize Rouge
rouge = Rouge()

def fetch_movie_plot(movie_name):
    try:
        # Directly fetch the page for the movie
        page = wikipedia.page(movie_name)
        content = page.content

        # Find the plot section and extract the content
        plot_section = ""
        start_idx = content.find("== Plot ==")

        if start_idx == -1:
            start_idx = content.find("== Plot==")  # Check for variations in section header

        if start_idx != -1:
            end_idx = content.find("\n==", start_idx + len("== Plot =="))
            if end_idx == -1:
                plot_section = content[start_idx:].strip()  # If end of section not found, take until end of content
            else:
                plot_section = content[start_idx:end_idx].strip()
        else:
            plot_section = content

        return plot_section

    except wikipedia.exceptions.PageError:
        return f"Page not found for '{movie_name}'. Please check the title and try again."
    except wikipedia.exceptions.DisambiguationError as e:
        # If there are multiple options, you can handle it here (e.g., select the first one)
        try:
            page = wikipedia.page(e.options[0])
            content = page.content

            plot_section = ""
            start_idx = content.find("== Plot ==")

            if start_idx == -1:
                start_idx = content.find("== Plot==")  # Check for variations in section header

            if start_idx != -1:
                end_idx = content.find("\n==", start_idx + len("== Plot =="))
                if end_idx == -1:
                    plot_section = content[start_idx:].strip()  # If end of section not found, take until end of content
                else:
                    plot_section = content[start_idx:end_idx].strip()
            else:
                plot_section = content

            return plot_section

        except wikipedia.exceptions.PageError:
            return f"Page not found for '{movie_name}'. Please check the title and try again."
        except wikipedia.exceptions.DisambiguationError:
            return f"Disambiguation error encountered for '{movie_name}'. Please refine your search."


def generate_summary(content, max_length=1024):
    if content and not content.startswith("Page not found"):
        # Tokenize the content into sentences using NLTK
        sentences = sent_tokenize(content)

        # Generate summary using T5
        inputs = tokenizer.encode("summarize: " + content, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs, max_length=900, min_length=100, length_penalty=5.0, num_beams=10)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Extract unique sentences from the generated summary
        unique_summary = []
        for sentence in sent_tokenize(summary):
            if sentence not in unique_summary:
                unique_summary.append(sentence)

        # Join unique sentences back into a summary
        final_summary = '. '.join(unique_summary)

        return final_summary
    else:
        return content

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Testing and evaluation
# BLEU-BiLingual Evaluation Understudy
# ROUGE-Recall-Oriented Understudy for Gisting Evaluation
from rouge import Rouge
import nltk
from nltk.tokenize import word_tokenize

# Initialize NLTK tokenizer and download punkt if necessary
nltk.download('punkt')

# Initialize ROUGE
rouge = Rouge()

# Function to get plot summary
def get_plot_summary(movie_title, df):
    summary = df.loc[df['TITLE'] == movie_title, 'Plot Summary(bart)']
    if not summary.empty:
        content = df.loc[df['TITLE'] == movie_title, 'PlotSummary'].values[0]
        return summary.values[0], content  # Return both summary and content
    else:
        wikipedia_summary = fetch_movie_plot(movie_title)
        if wikipedia_summary:
            t5_summary = generate_summary(wikipedia_summary)
            return t5_summary, wikipedia_summary
        else:
            return f"Plot summary for '{movie_title}' not found in dataset or Wikipedia.", None

# Main loop
while True:
    movie_title = input("\nEnter title of the movie to summarize (type 'exit' to quit):\n")
    if movie_title.lower() == 'exit':
        print("\nExited")
        break
    else:
        summary, content = get_plot_summary(movie_title, op)
        print(summary)

        if content and summary:
            # Calculate ROUGE-L scores
            rouge_scores = rouge.get_scores(summary, content, avg=True)

            # Print ROUGE-L scores
            print("\nROUGE-L Scores:")
            print(f"Precision:", rouge_scores['rouge-l']['p'])
            print(f"Recall:", rouge_scores['rouge-l']['r'])
            print(f"F1 Score:", rouge_scores['rouge-l']['f'])
        else:
            print("\nUnable to calculate ROUGE-L scores.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Enter title of the movie to summarize (type 'exit' to quit):
12 Angry Men
The 18-year-old boy is accused of killing his abusive father. A neighbor testified to witnessing the defendant stab his father, from her window, through the windows of a passing train. In a preliminary vote, all jurors vote "guilty" except Juror 8, who believes there should be some discussion before the verdict is made. Juror 3, infuriated, argues with and tries to attack Juror.

ROUGE-L Scores:
Precision: 0.6111111111111112
Recall: 0.5789473684210527
F1 Score: 0.5945945895982471

Enter title of the movie to summarize (type 'exit' to quit):
chhichhore
anirudh "anni" pathak is a divorced middle-aged man living with his teenage son.. he is awaiting the results of his entrance examination in the hope of enrolling in the iit-jEE.. he deliberately slips off the balcony to commit suicide, but survives, although in a fatal state.. Anni and his ex-wife maya narrates the tales of their college days in an attempt to rekin